<a href="https://colab.research.google.com/github/celelunar/News-Summarization/blob/main/News%20Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Diva Nabila Henryka

ID: 2501975620

## Install and Import Libraries

In [ ]:
!pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.4 MB/s eta 0:00:00


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=90f3da1ef9688a4017fa2716c9518d3b56a51cf7699ffdaf092a7ab3a6da4c6f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import pandas as pd
import numpy as np

import re

import nltk
from nltk import punkt
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.cluster.util import cosine_distance

import networkx as nx
from pyvis.network import Network

from rouge_score import rouge_scorer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Read the Data
The data is first uploaded into the Google Colaboratory File System before it can be read by the pandas library function `pd.read_csv()`

In [ ]:
df= pd.read_csv('data_3D.csv')
df

,Unnamed: 0,content,summary
0,14,"TEMPO.CO, Jakarta -Ketua Umum Partai Gerindra,...","Ketua Umum Partai Gerindra, Prabowo Subianto, ..."
1,15,"TEMPO.CO, Jakarta -Komisi Nasional Hak Asasi M...",Komisi Nasional Hak Asasi Manusia (Komnas HAM)...
2,16,"TEMPO.CO, Jakarta - Wacana Pemilu 2024 ditunda...",Wacana penundaan Pemilu 2024 kembali mencuat. ...


## Functions Building

### Text Preprocessing and Cleaning
Clean the news article header from each articles, then split it into sentences by using the dot operator. The result can then be saved into a list.

In [ ]:
def preprocess(text):

  # Remove the news article header
  text = re.sub(r'TEMPO.CO, Jakarta -', '', text)

  # Split the article into sentences using split by the dot operator
  sentences = re.split(r"\.", text)

  # Ensure that the sentences are splitted correctly
  # Save it to a list
  final = []
  for sentence in sentences:
      final.append(sentence.split())

  return final

### Check for Similarity Between Two Sentences
Cosine similarity itself is a metric used to determine how similar two vectors are by measuring the cosine of the angle between the two (finding the dot product and divide it by the product of vectors' magnitude).

This function first convert the sentences to lowercase and then remove any stopwords. The result will be saved in a list of all unique words in either sentence. The list is then used to create a vector for each sentence. Lastly, calculate the cosine similarity between the two sentences using the vectors.

In [ ]:
def similarity_checker(text1, text2, stopwords=None):

  # List of stopwords
  if stopwords is None:
    stopwords = []

  # Sentences to lower case
  text1 = [i.lower() for i in text1]
  text2 = [i.lower() for i in text2]

  # List containing words from both sentences
  words = list(set(text1 + text2))

  # Create vector 0 for each sentences
  vector1 = [0] * len(words)
  vector2 = [0] * len(words)

  # Edit vector for each sentence
  # Count the number of time a word appear on each sentence
  # Don't count if the word is a stopword
  for i in text1:
      if i in stopwords:
          continue
      vector1[words.index(i)] += 1

  for i in text2:
      if i in stopwords:
          continue
      vector2[words.index(i)] += 1

  # Avoid division by zero (error handling)
  if np.dot(vector1, vector1) == 0 or np.dot(vector2, vector2) == 0: return 0.0

  # Return cosine similarity
  return 1 - cosine_distance(vector1, vector2)

### Create Similarity Matrix
The function first creates an empty similarity matrix. The size of the matrix is equal to the number of sentences in the list of texts.

Next, the function iterates over the list of texts. For each pair of sentences, the function calculates the cosine similarity between the two sentences using the `similarity_checker` function that was created above. The cosine similarity is then saved in the similarity matrix.

In [ ]:
def similarity_matrix(texts, stopwords):

  # Empty matrix
  similarity_matrix = np.zeros((len(texts), len(texts)))

  # Iteratively check the similarity between sentences in the article
  # Ignoring if the iterative process i and j gives the same sentence
  # Save the cosine similarity to the empty matrix
  for i in range(len(texts)):
      for j in range(len(texts)):
          if i == j:
              continue
          similarity_matrix[i][j] = similarity_checker(texts[i], texts[j], stopwords)

  return similarity_matrix

### Text Summarization
This function is the core for this task which is to summarize articles given by the top 3 and 5 sentences in said articles. The 3 functions above are used in this function to preprocess the article and creating the similarity matrix.

The `nx.from_numpy_array()` function from the `networkx` library is used to convert the similarity matrix into a network graph. This graph represents sentences as nodes and their similarity as connections between them.

Next step is to rank the sentences using the `nx.pagerank()` function. PageRank assigns a higher score to sentences that are similar to many other high-scoring sentences. As error handling, if the PageRank calculation fails to converge, the function returns an empty list and an empty string.

The rank is then sorted from highest to lowest. When the function is called and given the `top_n` parameter, it will loops through the sorted sentences list and appends each sentence which acts as the final summary.

In [ ]:
def summarize(text, top_n):

  # Set stopwords to Indonesian
  stopwords = stopwords.words('indonesian')

  # Empty list for the summary result
  summarize_text = []

  # Call functions
  sentences = preprocess(text)
  article_similarity_matrix = similarity_matrix(sentences, stopwords)

  # Rank the sentences by the similarity matrix
  article_similarity_graph = nx.from_numpy_array(article_similarity_matrix)
  # Handle convergence failure by returning empty results (error handling)
  try:
      scores = nx.pagerank(article_similarity_graph)
  except nx.PowerIterationFailedConvergence:
      return [], ""

  # Rank sorting
  ranked = sorted(((scores[i], s) for i, s
                   in enumerate(sentences)), reverse=True)

  # Select the top n sentences
  # Save it as the summarized article result
  for i in range(top_n):
      summarize_text.append(" ".join(ranked[i][1]))

  # Return the rank and summary
  return ranked, ". ".join(summarize_text)

## Text Summarization Function Calling
This part of the code is used to call the summarization function and save the result to a dataframe by using loop. Since the question asked to do 2 summarization, which are using 3 and 5 top sentences, the function will be called 2 times.

In [ ]:
three_summary = pd.DataFrame(index = range(3),
                             columns = ['Ranked Scores', 'Summary'])
five_summary = pd.DataFrame(index = range(3),
                            columns = ['Ranked Scores', 'Summary'])

In [ ]:
for i in range(3):
  ranked, summary = summarize(df['content'][i], 3)
  three_summary.loc[i] = [ranked, summary]

  ranked, summary = summarize(df['content'][i], 5)
  five_summary.loc[i] = [ranked, summary]

In [ ]:
three_summary

,Ranked Scores,Summary
0,"[(0.07081064353876103, [Dalam, pertemuan, itu,...","Dalam pertemuan itu, Prabowo menyatakan bahwa ..."
1,"[(0.08322558561720768, [Hari, berujar, tim, Ko...",Hari berujar tim Komnas HAM belum menemukan in...
2,"[(0.04780008208922524, [Bahkan, pihaknya, meng...",Bahkan pihaknya mengatakan wacana penundaan Pe...


In [ ]:
five_summary

,Ranked Scores,Summary
0,"[(0.07081064353876103, [Dalam, pertemuan, itu,...","Dalam pertemuan itu, Prabowo menyatakan bahwa ..."
1,"[(0.08322558561720768, [Hari, berujar, tim, Ko...",Hari berujar tim Komnas HAM belum menemukan in...
2,"[(0.04780008208922524, [Bahkan, pihaknya, meng...",Bahkan pihaknya mengatakan wacana penundaan Pe...


## Summary Result

### First Row

#### Three Sentences Summary

In [ ]:
pd.DataFrame(three_summary['Ranked Scores'][0],
             columns = ['Score', 'Sentences'])

,Score,Sentences
0,0.070811,"[Dalam, pertemuan, itu,, Prabowo, menyatakan, ..."
1,0.067507,"[Kami, dapat, suatu, kesimpulan, untuk, sepaka..."
2,0.065863,"[Pertemuan, dua, sahabatKunjungan, Surya, Palo..."
3,0.065418,"[Prabowo, mengakui, adanya, pembicaraan, soal,..."
4,0.064358,"[Pada, pertemuan, Juni, lalu,, Prabowo, dan, S..."
5,0.058793,"[""Saya, mendapat, kehormatan, kunjungan, balas..."
6,0.057804,"[Prabowo, menyatakan, bahwa, pertemuan, terseb..."
7,0.053652,"[Ketua, Umum, Partai, Gerindra,, Prabowo, Subi..."
8,0.053622,"[Gerindra, di, satu, sisi, telah, mendeklarasi..."
9,0.048914,"[""Kita, sepakat, ini, komunikasi, politik, leb..."


In [ ]:
three_summary['Summary'][0]

'Dalam pertemuan itu, Prabowo menyatakan bahwa mereka sepakat untuk saling menghormati keputusan politik masing-masing. Kami dapat suatu kesimpulan untuk sepakat saling menghormati keputusan politik masing-masing," kata Prabowo usai pertemuan itu. Pertemuan dua sahabatKunjungan Surya Paloh ini merupakan balasan dari kunjungan Prabowo Subianto ke Kantor DPP NasDem pada Juni tahun lalu'

#### Five Sentences Summary

In [ ]:
pd.DataFrame(five_summary['Ranked Scores'][0], columns=['Score', 'Sentences'])

,Score,Sentences
0,0.070811,"[Dalam, pertemuan, itu,, Prabowo, menyatakan, ..."
1,0.067507,"[Kami, dapat, suatu, kesimpulan, untuk, sepaka..."
2,0.065863,"[Pertemuan, dua, sahabatKunjungan, Surya, Palo..."
3,0.065418,"[Prabowo, mengakui, adanya, pembicaraan, soal,..."
4,0.064358,"[Pada, pertemuan, Juni, lalu,, Prabowo, dan, S..."
5,0.058793,"[""Saya, mendapat, kehormatan, kunjungan, balas..."
6,0.057804,"[Prabowo, menyatakan, bahwa, pertemuan, terseb..."
7,0.053652,"[Ketua, Umum, Partai, Gerindra,, Prabowo, Subi..."
8,0.053622,"[Gerindra, di, satu, sisi, telah, mendeklarasi..."
9,0.048914,"[""Kita, sepakat, ini, komunikasi, politik, leb..."


In [ ]:
five_summary['Summary'][0]

'Dalam pertemuan itu, Prabowo menyatakan bahwa mereka sepakat untuk saling menghormati keputusan politik masing-masing. Kami dapat suatu kesimpulan untuk sepakat saling menghormati keputusan politik masing-masing," kata Prabowo usai pertemuan itu. Pertemuan dua sahabatKunjungan Surya Paloh ini merupakan balasan dari kunjungan Prabowo Subianto ke Kantor DPP NasDem pada Juni tahun lalu. Prabowo mengakui adanya pembicaraan soal politik dalam pertemuan itu. Pada pertemuan Juni lalu, Prabowo dan Surya masih membuka peluang Gerindra dan NasDem menjalin koalisi'

#### Analysis

To me, both summaries above are decent enough for the article and it can easily be read and understood. Though I would prefer it to have the information about the meet ("pertemuan") first, like what kind of meet it was, between whom, etc., before going into the detail of what happened in that meeting as it will be easier to be understood without the readers wondering what kind of meeting it was while reading the first few sentences.

### Second Row

#### Three Sentences Summary

In [ ]:
pd.DataFrame(three_summary['Ranked Scores'][1], columns=['Score', 'Sentences'])

,Score,Sentences
0,0.083226,"[Hari, berujar, tim, Komnas, HAM, belum, menem..."
1,0.065210,"[Aksi, tersebut, dilakukan, untuk, mendesak, K..."
2,0.061846,"[Namun,, kata, dia,, tim, sudah, menemukan, in..."
3,0.059119,"[Komisioner, Komnas, HAM, Hari, Kurniawan, men..."
4,0.057120,"[""Tim, investigasi, Komnas, HAM, ini, tidak, m..."
5,0.056753,"[Ihwal, perkembangan, investigasi,, Hari, meng..."
6,0.051894,"[""Belum,, kalau, Komnas, HAM, belum, menemukan..."
7,0.051278,"[Komisi, Nasional, Hak, Asasi, Manusia, akan, ..."
8,0.045950,"[Pasca, kejadian, tersebut,, sejumlah, mahasis..."
9,0.043146,"[Mengenai, dugaan, adanya, penembakan, oleh, a..."


In [ ]:
three_summary['Summary'][1]

'Hari berujar tim Komnas HAM belum menemukan indikasi penculikan. Aksi tersebut dilakukan untuk mendesak Komnas HAM menerjunkan tim untuk menyelidiki kasus tersebut. Namun, kata dia, tim sudah menemukan indikasi kekerasan aparat terhadap masyarakat sipil'

#### Five Sentences Summary

In [ ]:
pd.DataFrame(five_summary['Ranked Scores'][1], columns=['Score', 'Sentences'])

,Score,Sentences
0,0.083226,"[Hari, berujar, tim, Komnas, HAM, belum, menem..."
1,0.065210,"[Aksi, tersebut, dilakukan, untuk, mendesak, K..."
2,0.061846,"[Namun,, kata, dia,, tim, sudah, menemukan, in..."
3,0.059119,"[Komisioner, Komnas, HAM, Hari, Kurniawan, men..."
4,0.057120,"[""Tim, investigasi, Komnas, HAM, ini, tidak, m..."
5,0.056753,"[Ihwal, perkembangan, investigasi,, Hari, meng..."
6,0.051894,"[""Belum,, kalau, Komnas, HAM, belum, menemukan..."
7,0.051278,"[Komisi, Nasional, Hak, Asasi, Manusia, akan, ..."
8,0.045950,"[Pasca, kejadian, tersebut,, sejumlah, mahasis..."
9,0.043146,"[Mengenai, dugaan, adanya, penembakan, oleh, a..."


In [ ]:
five_summary['Summary'][1]

'Hari berujar tim Komnas HAM belum menemukan indikasi penculikan. Aksi tersebut dilakukan untuk mendesak Komnas HAM menerjunkan tim untuk menyelidiki kasus tersebut. Namun, kata dia, tim sudah menemukan indikasi kekerasan aparat terhadap masyarakat sipil. Komisioner Komnas HAM Hari Kurniawan mengatakan tim tersebut independen dengan tidak ada unsur dari TNI dan Polri. "Tim investigasi Komnas HAM ini tidak melibatkan TNI-Polri karena mereka sebagai pihak yang diperiksa Komnas HAM konteks peristiwa Wamena," ujar dia pada Jum\'at 3 Maret 2023'

#### Analysis

This is worse than the first article summaries, since it does not even mention what kind of case that is being discussed in the article. This means that the readers have to read between the lines to fully understand the context.

### Third Row

#### Three Sentences Summary

In [ ]:
pd.DataFrame(three_summary['Ranked Scores'][2], columns=['Score', 'Sentences'])

,Score,Sentences
0,0.047800,"[Bahkan, pihaknya, mengatakan, wacana, penunda..."
1,0.040652,"[Saya, mengusulkan, Pemilu, 2024, ditunda, sat..."
2,0.039858,"[Wacana, Pemilu, 2024, ditunda, kembali, mencuat]"
3,0.039257,"[Wakil, Ketua, DPR, RI, itu, mengusulkan, agar..."
4,0.033388,"[PLt, Ketum, PPP, Muhammad, MardionoWacana, pe..."
5,0.032121,"[Ketum, PAN, Zulkifli, HasanKetum, Partai, Ama..."
6,0.031107,"[Pilihan, Editor:, Penundaan, Pemilu, 2024, Bi..."
7,0.030774,"[Berikut, sejumlah, pihak, yang, pernah, mengg..."
8,0.029451,"[Ketum, PKB, Muhaimin, IskandarWacana, penunda..."
9,0.029144,"[Menkomarves, Luhut, Binsar, PandjaitanMenteri..."


In [ ]:
three_summary['Summary'][2]

'Bahkan pihaknya mengatakan wacana penundaan Pemilu 2024 tersebut merupakan aspirasi dari masyarakat. Saya mengusulkan Pemilu 2024 ditunda satu atau dua tahun. Wacana Pemilu 2024 ditunda kembali mencuat'

#### Five Sentences Summary

In [ ]:
pd.DataFrame(five_summary['Ranked Scores'][2], columns=['Score', 'Sentences'])

,Score,Sentences
0,0.047800,"[Bahkan, pihaknya, mengatakan, wacana, penunda..."
1,0.040652,"[Saya, mengusulkan, Pemilu, 2024, ditunda, sat..."
2,0.039858,"[Wacana, Pemilu, 2024, ditunda, kembali, mencuat]"
3,0.039257,"[Wakil, Ketua, DPR, RI, itu, mengusulkan, agar..."
4,0.033388,"[PLt, Ketum, PPP, Muhammad, MardionoWacana, pe..."
5,0.032121,"[Ketum, PAN, Zulkifli, HasanKetum, Partai, Ama..."
6,0.031107,"[Pilihan, Editor:, Penundaan, Pemilu, 2024, Bi..."
7,0.030774,"[Berikut, sejumlah, pihak, yang, pernah, mengg..."
8,0.029451,"[Ketum, PKB, Muhaimin, IskandarWacana, penunda..."
9,0.029144,"[Menkomarves, Luhut, Binsar, PandjaitanMenteri..."


In [ ]:
five_summary['Summary'][2]

'Bahkan pihaknya mengatakan wacana penundaan Pemilu 2024 tersebut merupakan aspirasi dari masyarakat. Saya mengusulkan Pemilu 2024 ditunda satu atau dua tahun. Wacana Pemilu 2024 ditunda kembali mencuat. Wakil Ketua DPR RI itu mengusulkan agar Pemilu 2024 ditunda selama satu atau dua tahun. PLt Ketum PPP Muhammad MardionoWacana penundaan pemilu 2024 kembali bergulir pada awal Februari lalu'

#### Analysis

I would definitely said that this one is the best out of all three since I can immediately tell the main topic of the article from the first sentence.

## ROUGE Score
ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate the quality of automatic summaries of text, and also machine translations. It works by comparing a generated summary to a reference summary (written by a human) and measures how similar they are. It ranges from 0 to 1, where higher scores indicate a better summary.

There will be 3 types of ROUGE scores that will be used here:
- ROUGE-N: measures the overlap in n grams (sequence of n words).
  - ROUGE-1: considers unigrams (single words).
  - ROUGE-2: considers bigrams (pair of words).
- ROUGE-L: measures the length of the longest matching sequence.

What each metrics mean:
- Precision: the level of relevant information in the summary.
- Recall: the captured important keywords and information.
- F1 score: balance between precision and recall.

In [ ]:
rouge_score = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

In [ ]:
score_three = []
score_five = []

for i in range(3):
  score_three.append(rouge_score.score(three_summary['Summary'][i], df['summary'][i]))
  score_five.append(rouge_score.score(five_summary['Summary'][i], df['summary'][i]))

In [ ]:
def print_rouge_scores(scores, summary_name):
    for i, score in enumerate(scores):
        print(f"\n{summary_name} Summary {i+1}:")
        for key, value in score.items():
            print(f"  {key}:")
            print(f"    Precision: {value.precision:.4f}")
            print(f"    Recall:    {value.recall:.4f}")
            print(f"    F1 Score:  {value.fmeasure:.4f}")

### Three Sentences Summary

In [ ]:
# Print the scores for three_summary and five_summary
print("ROUGE Scores for three_summary:")
print_rouge_scores(score_three, "Three Sentences")

ROUGE Scores for three_summary:

Three Sentences Summary 1:
  rouge1:
    Precision: 0.2712
    Recall:    0.3077
    F1 Score:  0.2883
  rouge2:
    Precision: 0.1379
    Recall:    0.1569
    F1 Score:  0.1468
  rougeL:
    Precision: 0.2203
    Recall:    0.2500
    F1 Score:  0.2342

Three Sentences Summary 2:
  rouge1:
    Precision: 0.3077
    Recall:    0.3529
    F1 Score:  0.3288
  rouge2:
    Precision: 0.0526
    Recall:    0.0606
    F1 Score:  0.0563
  rougeL:
    Precision: 0.1795
    Recall:    0.2059
    F1 Score:  0.1918

Three Sentences Summary 3:
  rouge1:
    Precision: 0.3158
    Recall:    0.4444
    F1 Score:  0.3692
  rouge2:
    Precision: 0.1892
    Recall:    0.2692
    F1 Score:  0.2222
  rougeL:
    Precision: 0.2368
    Recall:    0.3333
    F1 Score:  0.2769


#### Analysis

**First Article**
- Compared to the second and third summary, the first one is the lowest in all ROUGE-1 metrics. But it is the second best for the ROUGE-2 and ROUGE-L metrics.
- It has a better **balance between including relevant information and avoiding irrelevant content**, reflected in higher F1 scores across all metrics.

**Second Article**
- Even though the ROUGE-1 metrics is better than the first summary, this second article's summary has the lowest score for all ROUGE-2 and ROUGE-L metrics.
- Lower scores in ROUGE-2 and ROUGE-L suggest a **poorer match in phrasing and overall content compared to the reference**.

**Third Article**
- This summary is the most consistent across all type of ROUGE scores with each one being the highest score compared to the other articles.
- It captures most of the important single words from the reference and **maintains a good balance between including relevant information and avoiding irrelevant ones**.

### Five Sentences Summary

In [ ]:
print("ROUGE Scores for five_summary:")
print_rouge_scores(score_five, "Five Sentences")

ROUGE Scores for five_summary:

Five Sentences Summary 1:
  rouge1:
    Precision: 0.3729
    Recall:    0.2895
    F1 Score:  0.3259
  rouge2:
    Precision: 0.1379
    Recall:    0.1067
    F1 Score:  0.1203
  rougeL:
    Precision: 0.2203
    Recall:    0.1711
    F1 Score:  0.1926

Five Sentences Summary 2:
  rouge1:
    Precision: 0.5385
    Recall:    0.2658
    F1 Score:  0.3559
  rouge2:
    Precision: 0.1842
    Recall:    0.0897
    F1 Score:  0.1207
  rougeL:
    Precision: 0.3333
    Recall:    0.1646
    F1 Score:  0.2203

Five Sentences Summary 3:
  rouge1:
    Precision: 0.4474
    Recall:    0.3036
    F1 Score:  0.3617
  rouge2:
    Precision: 0.3243
    Recall:    0.2182
    F1 Score:  0.2609
  rougeL:
    Precision: 0.3684
    Recall:    0.2500
    F1 Score:  0.2979


#### Analysis

First Article
- Shows a moderate performance where it has a decent precision, indicating it **avoids too much irrelevant information**.
- But the recall is lower, suggesting it might **miss some important points** from the reference summary.

Second Article
- It has a higher precision scores compared to the first summary, which means it **includes a higher proportion of relevant information**.
- However, the recall is lower, suggesting it **might miss more key points**.

Third Article
- This summary achieves a good balance between precision and recall, reflected in the F1 scores.
- It **captures a significant portion of important information** (good recall) **while avoiding excessive irrelevant content** (good precision).

## Summary
3-Sentences Summary:
- Summary 3 has the highest overall ROUGE scores, suggesting the best match to the reference summary.
- Summary 1 shows improvement over Summary 2 but doesn't quite reach the level of Summary 3.
- Summary 2 has the lowest scores, indicating a weaker match in terms of both content and phrasing.

5-Sentences Summary:
- Summary 3 again achieves a good balance between precision and recall (reflected in F1 scores), suggesting it effectively captures relevant information.
- Summary 2 has the highest precision, indicating a concise summary focused on key points, but it might miss some details compared to Summary 3.
- Summary 1 shows a moderate performance with a trade-off between including relevant information and missing important details.